## Import labiries

In [1]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time
import collections
import re
import pickle
from random import randint
from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from six.moves import cPickle
from IPython.display import Image

## Process data

In [2]:
import string

#load text into memeory
def load_text(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

In [3]:
#turn text into clean tokens
def clean_text(text):
    # replace '--' with a space ' '
    text = text.replace('--', ' ')
    # split into tokens by white space
    tokens = text.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # make lower case
    tokens = [word.lower() for word in tokens]
    return tokens

In [4]:
#save tokens
def save_text(lines,filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()
 

In [5]:
#load text
in_filename = 'short_jokes.txt'
text = load_text(in_filename)
print(text[:1000])

I was eating at an Indian restaurant when a homeless man came in and asked the girl behind the counter if they had any food he could have. She told him there was Naan. [me narrating a documentary about narrators] "I can't hear what they're saying cuz I'm talking"
I was going to organize a space-themed birthday party for my son... ...but I couldn't planet. Telling my daughter garlic is good for you. Good immune system and keeps pests away.Ticks, mosquitos, vampires... men.
Squared My dick is X squared and I rooted your mom. Now she's my X I've been going through a really rough period at work this week It's my own fault for swapping my tampax for sand paper.
I call my dick fun Because it's what girls just want to have! If I could have dinner with anyone, dead or alive... ...I would choose alive. -B.J. Novak-
Be careful when you ROFL! I once heard a joke in a scissor factory... It left me in stitches. Two guys walk into a bar. The third guy ducks.
I'll bet Vampire Kiddies enjoy scabs as m

In [6]:
#clean text
tokens = clean_text(text)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

['i', 'was', 'eating', 'at', 'an', 'indian', 'restaurant', 'when', 'a', 'homeless', 'man', 'came', 'in', 'and', 'asked', 'the', 'girl', 'behind', 'the', 'counter', 'if', 'they', 'had', 'any', 'food', 'he', 'could', 'have', 'she', 'told', 'him', 'there', 'was', 'naan', 'me', 'narrating', 'a', 'documentary', 'about', 'narrators', 'i', 'cant', 'hear', 'what', 'theyre', 'saying', 'cuz', 'im', 'talking', 'i', 'was', 'going', 'to', 'organize', 'a', 'spacethemed', 'birthday', 'party', 'for', 'my', 'son', 'but', 'i', 'couldnt', 'planet', 'telling', 'my', 'daughter', 'garlic', 'is', 'good', 'for', 'you', 'good', 'immune', 'system', 'and', 'keeps', 'pests', 'awayticks', 'mosquitos', 'vampires', 'men', 'squared', 'my', 'dick', 'is', 'x', 'squared', 'and', 'i', 'rooted', 'your', 'mom', 'now', 'shes', 'my', 'x', 'ive', 'been', 'going', 'through', 'a', 'really', 'rough', 'period', 'at', 'work', 'this', 'week', 'its', 'my', 'own', 'fault', 'for', 'swapping', 'my', 'tampax', 'for', 'sand', 'paper', 'i

In [7]:
#organize into sequences of tokens
length = 50 + 1
sequences = list()
for i in range(length,len(tokens)):
    # select sequence of tokens
    seq = tokens[i-length:i]
    # convert into a line
    line = ' '.join(seq)
    # store
    sequences.append(line)
print('Total Sequences: %d' % len(sequences))
 
# save sequences to file
out_filename = 'scifi_sequences.txt'
save_text(sequences, out_filename)

Total Sequences: 158288


In [8]:
in_filename = 'scifi_sequences.txt'
text = load_text(in_filename)
lines = text.split('\n')


In [9]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
#vocabulary size
vocab_size = len(tokenizer.word_index)+1

In [10]:
#separate into input and output
sequences = np.array(sequences)
x,y = sequences[:,:-1], sequences[:,-1]
y = tf.keras.utils.to_categorical(y,num_classes=vocab_size)
seq_length = x.shape[1]

## Define and Build Model

In [11]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size,50,input_length=seq_length))
model.add(tf.keras.layers.LSTM(100,return_sequences=True))
model.add(tf.keras.layers.LSTM(100))
model.add(tf.keras.layers.Dense(100, activation='relu'))
model.add(tf.keras.layers.Dense(vocab_size,activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 50)            706200    
_________________________________________________________________
lstm (LSTM)                  (None, 50, 100)           60400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 14124)             1426524   
Total params: 2,283,624
Trainable params: 2,283,624
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
#compile model
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
#fit model
model.fit(x,y,batch_size=64,epochs=30)
#save the model to file
model.save('model.h5')
#save the tokenizer
pickle.dump(tokenizer,open('tokenizer.pkl','wb'))

W1018 23:18:00.362707 140715490535232 deprecation.py:323] From /opt/conda/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/30
158288/158288 [==============================] - 969s 6ms/sample - loss: 6.9889 - acc: 0.0414
Epoch 2/30
158288/158288 [==============================] - 980s 6ms/sample - loss: 6.6026 - acc: 0.0605
Epoch 3/30
158288/158288 [==============================] - 980s 6ms/sample - loss: 6.3585 - acc: 0.0748
Epoch 4/30
158288/158288 [==============================] - 966s 6ms/sample - loss: 6.1656 - acc: 0.0912
Epoch 5/30
158288/158288 [==============================] - 965s 6ms/sample - loss: 6.0064 - acc: 0.1026
Epoch 6/30
158288/158288 [==============================] - 952s 6ms/sample - loss: 5.8629 - acc: 0.1104
Epoch 7/30
158288/158288 [==============================] - 934s 6ms/sample - loss: 5.7278 - acc: 0.1182
Epoch 8/30
158288/158288 [==============================] - 943s 6ms/sample - loss: 5.6074 - acc: 0.1253
Epoch 9/30
158288/158288 [==============================] - 941s 6ms/sample - loss: 5.4983 - acc: 0.1316
Epoch 10/30
158288/158288 [============================

In [19]:
def generate_seq(model,tokenizer,seq_length,seed_text,n_words):
    result=list()
    in_text = seed_text
    #generate a fixed number of words
    for _ in range(n_words):
        #encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        #truncate sequences to a fixed length
        encoded = pad_sequences([encoded],maxlen=seq_length,truncating='pre')
        #predict probabilities for each word
        yhat=model.predict_classes(encoded, verbose=0)
        #map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        #append ti input
        in_text +=''+ out_word
        result.append(out_word)
    return ''.join(result)

#load cleaned text sequences
in_filename = 'scifi_sequences.txt'
text = load_text(in_filename)
lines = text.split('\n')
seq_length = len(lines[0].split())-1

#load model
model = tf.keras.models.load_model('model.h5')

#load the tokenizer
tokenizer =pickle.load(open('tokenizer.pkl','rb'))

#select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text +'\n')

#generate new text
generated = generate_seq(model,tokenizer,seq_length, seed_text, 50)


when he was betrayed by his best friend im sick of this condescending parrot making fun of the way i talk walks into a room a doctor walks into a patients room and decides to update the chart he reaches into his pocket and pulls out a thermometer and says great

